In [1]:
#instalo librerias faltantes

!pip install tensorflow
!pip install PyPDF2
!pip install fpdf
!pip install reportlab

#!pip install reportlab pillow
#!pip install tabulate

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from fpdf import FPDF
import PyPDF2
from PyPDF2 import PdfWriter, PdfReader
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from PIL import Image
from datetime import datetime
import tensorflow as tf
import smtplib
from email.message import EmailMessage
import os

2024-06-08 18:36:50.486303: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 18:36:50.486375: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 18:36:50.488127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
#lectura de data set
df_transactions = pd.read_parquet("/kaggle/input/transactions-train/transactions.parquet")

In [4]:
df_transactions.head(10)

,V15,V16,V17,V18,V21,V22,V23,V24,V29,V30,...,R_emaildomain_web.de,R_emaildomain_windstream.net,R_emaildomain_yahoo.co.jp,R_emaildomain_yahoo.co.uk,R_emaildomain_yahoo.com,R_emaildomain_yahoo.com.mx,R_emaildomain_yahoo.de,R_emaildomain_yahoo.es,R_emaildomain_yahoo.fr,R_emaildomain_ymail.com
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_transactions.columns

Index(['V15', 'V16', 'V17', 'V18', 'V21', 'V22', 'V23', 'V24', 'V29', 'V30',
       ...
       'R_emaildomain_web.de', 'R_emaildomain_windstream.net',
       'R_emaildomain_yahoo.co.jp', 'R_emaildomain_yahoo.co.uk',
       'R_emaildomain_yahoo.com', 'R_emaildomain_yahoo.com.mx',
       'R_emaildomain_yahoo.de', 'R_emaildomain_yahoo.es',
       'R_emaildomain_yahoo.fr', 'R_emaildomain_ymail.com'],
      dtype='object', length=166)

In [6]:
# Calculate the correlation matrix
corr_matrix = df_transactions.corr()

# Exclude the variable 'isFraud' from the correlation calculation
corr_with_target = corr_matrix.drop('isFraud')['isFraud'].dropna().sort_values(ascending=False)

In [7]:
# Split the correlation values into positive and negative values
positive_corr = corr_with_target[corr_with_target > 0].sort_values(ascending=False).to_frame('correlation').reset_index()
negative_corr = corr_with_target[corr_with_target < 0].sort_values(ascending=True).to_frame('correlation').reset_index()

# Rename the columns to 'variable' and 'correlation_with_isFraud'
positive_corr.columns = ['variable', 'correlation_with_isFraud']
negative_corr.columns = ['variable', 'correlation_with_isFraud']

# Define the output paths for the CSV files
positive_corr_csv_path = "/kaggle/working/positive_correlations.csv"
negative_corr_csv_path = "/kaggle/working/negative_correlations.csv"

# Save the correlation DataFrames to CSV files
positive_corr.to_csv(positive_corr_csv_path, index=False)
negative_corr.to_csv(negative_corr_csv_path, index=False)

print(f"Positive correlations have been exported to {positive_corr_csv_path} successfully.")
print(f"Negative correlations have been exported to {negative_corr_csv_path} successfully.")

Positive correlations have been exported to /kaggle/working/positive_correlations.csv successfully.
Negative correlations have been exported to /kaggle/working/negative_correlations.csv successfully.


In [8]:
# Split the correlation values into positive and negative values for doing top 5
positive_corr = corr_with_target[corr_with_target > 0].sort_values(ascending=False).head(5).to_frame('Correlation')
negative_corr = corr_with_target[corr_with_target < 0].sort_values(ascending=True).head(5).to_frame('Correlation')

# Format the correlation values with three decimal places
positive_corr['Correlation'] = positive_corr['Correlation'].apply(lambda x: f"{x:.3f}")
negative_corr['Correlation'] = negative_corr['Correlation'].apply(lambda x: f"{x:.3f}")

# Define data and titles
data = [positive_corr, negative_corr]
titles = ['Top 5 variables with positive correlation', 'Top 5 variables with negative correlation']

# Iterate through data and save each table as an image
for df, title in zip(data, titles):
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.axis('off')
    ax.set_title(title)
    ax.table(cellText=df.values,
             colLabels=['Correlation'],
             rowLabels=df.index,
             loc='center',
             cellLoc='center',
             colColours=['lightgray'],
             rowColours=['lightgray'] * len(df),
             bbox=[0, 0, 1, 1])
    plt.tight_layout()
    plt.savefig(f'{title}.png')
    plt.close()

print("Positive and Negetive correlation generated successfully.")

Positive and Negetive correlation generated successfully.


In [9]:
# Preprocesamiento de datos
X = df_transactions.drop(['isFraud'], axis=1)
y = df_transactions['isFraud']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


# Evaluating the model
y_pred = rf_model.predict(X_test)
classification_rep = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [10]:
# Convert the confusion matrix to a DataFrame for easier handling
cm_df = pd.DataFrame(conf_matrix, index=['Actual Negative', 'Actual Positive'], columns=['Predicted Negative', 'Predicted Positive'])

# Define the output path for the CSV file
csv_output_path = "/kaggle/working/confusion_matrix.csv"

# Save the confusion matrix DataFrame to a CSV file
cm_df.to_csv(csv_output_path)

print(f"Confusion matrix has been exported to {csv_output_path} successfully.")

Confusion matrix has been exported to /kaggle/working/confusion_matrix.csv successfully.


In [11]:
# Create a grid of numbers for the confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
classes = ["Negative", "Positive"]
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, format(conf_matrix[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if conf_matrix[i, j] > conf_matrix.max() / 2 else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()

# Save the plot as an image
plt.savefig('/kaggle/working/confusion_matrix.png')
plt.close()

print("Confusion matrix generated successfully.")

Confusion matrix generated successfully.


In [12]:
# Paths to images and PDF
logo_path = "/kaggle/input/logo-project/WS.png"
confusion_matrix_path = "/kaggle/working/confusion_matrix.png"
pdf_file_path = "page_1.pdf"

# Open the logo image to get its dimensions
with Image.open(logo_path) as img:
    img_width, img_height = img.size

# Desired logo width in the PDF
logo_width = 100
# Calculate the corresponding logo height to preserve the aspect ratio
logo_height = logo_width * (img_height / img_width)

# Create a canvas
c = canvas.Canvas(pdf_file_path, pagesize=A4)
width, height = A4

# Define margins
top_margin = 20
left_margin = 20
right_margin = 20
bottom_margin = 20

# Calculate the position of the logo to center it horizontally at the top of the page
x = (width - logo_width) / 2
y = height - top_margin - logo_height

# Draw borders
c.line(left_margin, height - top_margin, width - right_margin, height - top_margin)
c.line(left_margin, height - top_margin, left_margin, bottom_margin)
c.line(left_margin, bottom_margin, width - right_margin, bottom_margin)
c.line(width - right_margin, height - top_margin, width - right_margin, bottom_margin)

# Draw the logo border lines
c.line(x, y + logo_height, x + logo_width, y + logo_height)
c.line(x, y + logo_height, x, y)
c.line(x + logo_width, y + logo_height, x + logo_width, y)
c.line(x, y, x + logo_width, y)

# Draw the logo
c.drawImage(logo_path, x, y, width=logo_width, height=logo_height, mask='auto')

# Get the current date and time in DD-MM-YYYY HH:MM:SS format
current_datetime = datetime.now().strftime('%d-%m-%Y %H:%M:%S')

# Draw the date and time to the left of the logo, 10 units from the left margin
text_x_left = left_margin + 10
text_y = y + (logo_height / 2)

c.setFont("Helvetica", 12)
c.drawString(text_x_left, text_y, "Execution date: " + current_datetime)

# Draw the creator name to the right of the logo, 10 units from the right margin
text_x_right = width - right_margin - 150  # Adjusted based on approximate text width
c.drawString(text_x_right, text_y, "Team: c18-80-t-data-bi")

# Draw the Confusion Matrix Explanation title in bold
c.setFont("Helvetica-Bold", 14)
c.drawString(left_margin, height - top_margin - 150, "Confusion Matrix Explanation")

# Define text content
explanation_text = [
    "It provides insight into the performance of the model by summarizing the counts of",
    "true positive (TP), true negative (TN), false positive (FP)",
    "and false negative (FN) predictions.",
    " ",
    "Interpretation:",
    "- True Negative (TN): instances were correctly classified as negative.",
    "- False Positive (FP): instances were incorrectly classified as positive.",
    "- False Negative (FN): instances were incorrectly classified as negative.",
    "- True Positive (TP): instances were correctly classified as positive.",
    " ",
    "Utilization:",
    "- Accuracy: The overall accuracy of the model can be calculated as the ratio",
    "of correct predictions (TP + TN) to the total number of instances.",
    "- Precision: Precision measures the proportion of true positive",
    "predictions among all positive predictions (TP / (TP + FP)).",
    "- Recall (Sensitivity): Recall calculates the proportion of",
    "true positive predictions among all actual positives (TP / (TP + FN)).",
    "- F1-Score: The F1-score, the harmonic mean of precision and",
    "recall, provides a balanced measure of a model's performance."
]

# Set font to normal for the explanation text
c.setFont("Helvetica", 12)

# Draw explanation text
text_x = left_margin + 10
text_y = height - top_margin - 180
line_height = 14

for line in explanation_text:
    c.drawString(text_x, text_y, line)
    text_y -= line_height

# Position of the confusion matrix image
space_between_text_and_image = 10
image_y = text_y - space_between_text_and_image - 300  # Adjust based on the height of the image

# Draw the confusion matrix image
c.drawImage(confusion_matrix_path, left_margin + 10, image_y, width=width - 2 * left_margin - 20, height=300, mask='auto')

# Finalize the PDF
c.showPage()
c.save()

print("Page #1 generated successfully.")

Page #1 generated successfully.


In [13]:
# Paths to images and PDF
logo_path = "/kaggle/input/logo-project/WS.png"
positive_correlation_path = "/kaggle/working/Top 5 variables with positive correlation.png"
pdf_file_path = "page_2.pdf"

# Open the logo image to get its dimensions
with Image.open(logo_path) as img:
    img_width, img_height = img.size

# Desired logo width in the PDF
logo_width = 100
# Calculate the corresponding logo height to preserve the aspect ratio
logo_height = logo_width * (img_height / img_width)

# Create a canvas
c = canvas.Canvas(pdf_file_path, pagesize=A4)
width, height = A4

# Define margins
top_margin = 20
left_margin = 20
right_margin = 20
bottom_margin = 20

# Calculate the position of the logo to center it horizontally at the top of the page
x = (width - logo_width) / 2
y = height - top_margin - logo_height

# Draw borders
c.line(left_margin, height - top_margin, width - right_margin, height - top_margin)
c.line(left_margin, height - top_margin, left_margin, bottom_margin)
c.line(left_margin, bottom_margin, width - right_margin, bottom_margin)
c.line(width - right_margin, height - top_margin, width - right_margin, bottom_margin)

# Draw the logo border lines
c.line(x, y + logo_height, x + logo_width, y + logo_height)
c.line(x, y + logo_height, x, y)
c.line(x + logo_width, y + logo_height, x + logo_width, y)
c.line(x, y, x + logo_width, y)

# Draw the logo
c.drawImage(logo_path, x, y, width=logo_width, height=logo_height, mask='auto')

# Get the current date and time in DD-MM-YYYY HH:MM:SS format
current_datetime = datetime.now().strftime('%d-%m-%Y %H:%M:%S')

# Draw the date and time to the left of the logo, 10 units from the left margin
text_x_left = left_margin + 10
text_y = y + (logo_height / 2)

c.setFont("Helvetica", 12)
c.drawString(text_x_left, text_y, "Execution date: " + current_datetime)

# Draw the creator name to the right of the logo, 10 units from the right margin
text_x_right = width - right_margin - 150  # Adjusted based on approximate text width
c.drawString(text_x_right, text_y, "Team: c18-80-t-data-bi")

# Draw the title "Top 5 variables with positive correlation"
c.setFont("Helvetica-Bold", 14)
title_y = height - top_margin - 150
c.drawString(left_margin, title_y, "Top 5 variables with positive correlation")

# Load the positive correlation image to get its dimensions
with Image.open(positive_correlation_path) as img:
    img_width, img_height = img.size

# Calculate the new image width and height
max_image_width = width - 2 * left_margin - 20  # Maximum width based on page margins
max_image_height = title_y - bottom_margin - 20  # Maximum height based on title and bottom margin
scale_factor = min(max_image_width / img_width, max_image_height / img_height)
new_image_width = img_width * scale_factor
new_image_height = img_height * scale_factor

# Calculate the position of the image to center it horizontally
image_x = (width - new_image_width) / 2
image_y = title_y - new_image_height - 20  # Position 20 units below the title

# Draw the positive correlation image
c.drawImage(positive_correlation_path, image_x, image_y, width=new_image_width, height=new_image_height, mask='auto')

# Finalize the PDF
c.showPage()
c.save()

print("Page #2 generated successfully.")



Page #2 generated successfully.


In [14]:
# Paths to images and PDF
logo_path = "/kaggle/input/logo-project/WS.png"
positive_correlation_path = "/kaggle/working/Top 5 variables with negative correlation.png"
pdf_file_path = "page_3.pdf"

# Open the logo image to get its dimensions
with Image.open(logo_path) as img:
    img_width, img_height = img.size

# Desired logo width in the PDF
logo_width = 100
# Calculate the corresponding logo height to preserve the aspect ratio
logo_height = logo_width * (img_height / img_width)

# Create a canvas
c = canvas.Canvas(pdf_file_path, pagesize=A4)
width, height = A4

# Define margins
top_margin = 20
left_margin = 20
right_margin = 20
bottom_margin = 20

# Calculate the position of the logo to center it horizontally at the top of the page
x = (width - logo_width) / 2
y = height - top_margin - logo_height

# Draw borders
c.line(left_margin, height - top_margin, width - right_margin, height - top_margin)
c.line(left_margin, height - top_margin, left_margin, bottom_margin)
c.line(left_margin, bottom_margin, width - right_margin, bottom_margin)
c.line(width - right_margin, height - top_margin, width - right_margin, bottom_margin)

# Draw the logo border lines
c.line(x, y + logo_height, x + logo_width, y + logo_height)
c.line(x, y + logo_height, x, y)
c.line(x + logo_width, y + logo_height, x + logo_width, y)
c.line(x, y, x + logo_width, y)

# Draw the logo
c.drawImage(logo_path, x, y, width=logo_width, height=logo_height, mask='auto')

# Get the current date and time in DD-MM-YYYY HH:MM:SS format
current_datetime = datetime.now().strftime('%d-%m-%Y %H:%M:%S')

# Draw the date and time to the left of the logo, 10 units from the left margin
text_x_left = left_margin + 10
text_y = y + (logo_height / 2)

c.setFont("Helvetica", 12)
c.drawString(text_x_left, text_y, "Execution date: " + current_datetime)

# Draw the creator name to the right of the logo, 10 units from the right margin
text_x_right = width - right_margin - 150  # Adjusted based on approximate text width
c.drawString(text_x_right, text_y, "Team: c18-80-t-data-bi")

# Draw the title "Top 5 variables with negative correlation"
c.setFont("Helvetica-Bold", 14)
title_y = height - top_margin - 150
c.drawString(left_margin, title_y, "Top 5 variables with negative correlation")

# Load the positive correlation image to get its dimensions
with Image.open(positive_correlation_path) as img:
    img_width, img_height = img.size

# Calculate the new image width and height
max_image_width = width - 2 * left_margin - 20  # Maximum width based on page margins
max_image_height = title_y - bottom_margin - 20  # Maximum height based on title and bottom margin
scale_factor = min(max_image_width / img_width, max_image_height / img_height)
new_image_width = img_width * scale_factor
new_image_height = img_height * scale_factor

# Calculate the position of the image to center it horizontally
image_x = (width - new_image_width) / 2
image_y = title_y - new_image_height - 20  # Position 20 units below the title

# Draw the positive correlation image
c.drawImage(positive_correlation_path, image_x, image_y, width=new_image_width, height=new_image_height, mask='auto')

# Finalize the PDF
c.showPage()
c.save()

print("Page #3 generated successfully.")

Page #3 generated successfully.


In [15]:
def concatenate_pdfs(pdf_paths, output_path):
    pdf_writer = PdfWriter()
    
    for pdf_path in pdf_paths:
        pdf_reader = PdfReader(pdf_path)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            pdf_writer.add_page(page)
    
    with open(output_path, 'wb') as output_pdf:
        pdf_writer.write(output_pdf)

# List of PDF file paths to concatenate
pdf_paths = [
    "/kaggle/working/page_1.pdf",  # Path to the first PDF
    "/kaggle/working/page_2.pdf",  # Path to the second PDF
    "/kaggle/working/page_3.pdf"
]

# Output path for the concatenated PDF
output_path = "/kaggle/working/Inform_ML_Model.pdf"

# Concatenate the PDF files
concatenate_pdfs(pdf_paths, output_path)

print("Inform ML Model generated successfully.")


Inform ML Model generated successfully.


In [17]:
# Email credentials and settings
smtp_server = 'smtp.gmail.com'
smtp_port = 587
sender_email = 'sender_email@gmail.com' #email's sender
sender_password = 'xxxx xxxx xxxx xxxx' #email's sender, which is generated using GMAIL
recipient_email = 'recipient_email@gmail.com' #email's recipient
subject = 'Enter subject of e-mail' #email's subject
body = 'You write body of e-mail when it will be the attached PDF file.' #email's body text

# Path to the PDF file
pdf_file_path = '/kaggle/working/Inform_ML_Model.pdf'

# Create the email message
msg = EmailMessage()
msg['Subject'] = subject
msg['From'] = sender_email
msg['To'] = recipient_email
msg.set_content(body)

# Attach the PDF file
with open(pdf_file_path, 'rb') as f:
    pdf_data = f.read()
    msg.add_attachment(pdf_data, maintype='application', subtype='pdf', filename=os.path.basename(pdf_file_path))

# Send the email
try:
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()  # Secure the connection
        server.login(sender_email, sender_password)
        server.send_message(msg)
    print('Email sent successfully!')
except Exception as e:
    print(f'Failed to send email: {e}')


Email sent successfully!
